In [8]:
import matplotlib.pyplot as plt


In [ ]:
import robotic as ry
import time
max_trials = 10
max_vel = 100
max_acc = max_vel

C = ry.Config()
C.addFile('/home/vboxuser/Desktop/CS449/Migi/hw3/GFiles-HW3/environment.g')

# C.addFile('./environment.g')
# C.addFile(ry.raiPath('environment.g'))
# qHome = C.getJointState()
# print(qHome)
C.view()
print(ry.raiPath('environment.g'))

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/environment.g


### 1.1 Perception Module

In [9]:
print("qHome: ", C.getJointState())

qHome:  [ 0.  -0.5  0.  -2.   0.   2.  -0.5]
